# 07 XGBoost with KFold Cross-Validation and Optuna Tuning

This notebook trains an XGBoost regressor using KFold cross-validation and hyperparameter optimization with Optuna to predict `MarketValue_NextSeason`. Artifacts (model and study results) are saved to `../models` and `../artifacts`.

In [1]:
from __future__ import annotations

import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
from typing import Dict, List, Tuple

import json
import pickle

import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import optuna
from optuna.samplers import TPESampler

from xgboost import XGBRegressor

In [2]:
# keep things reproducible
RANDOM_SEED: int = 42
np.random.seed(RANDOM_SEED)

In [3]:
PROJECT_ROOT: Path = Path("..").resolve()
DATA_DIR: Path = PROJECT_ROOT / "data" / "interim" / "datasets_combined"
ARTIFACTS_DIR: Path = PROJECT_ROOT / "artifacts"
MODELS_DIR: Path = PROJECT_ROOT / "models"

In [4]:
for directory in [ARTIFACTS_DIR, MODELS_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

In [5]:
csv_path: Path = DATA_DIR / "all_teams_combined.csv"
df_raw: pd.DataFrame = pd.read_csv(csv_path)

In [6]:
def build_target(df_with_market_values: pd.DataFrame) -> pd.DataFrame:
    """Build `MarketValue_NextSeason` and drop rows with missing target. If missing, derive by player-season shift of MarketValueEuro."""
    if "MarketValue_NextSeason" not in df_with_market_values.columns:
        required: set[str] = {"Player", "Season", "MarketValueEuro"}
        if required.issubset(df_with_market_values.columns):
            tmp: pd.DataFrame = df_with_market_values.copy()
            tmp["Season"] = tmp["Season"].astype(str)
            season_order: dict[str, int] = {s: i for i, s in enumerate(sorted(tmp["Season"].unique()))}
            tmp["_season_order"] = tmp["Season"].map(season_order)
            tmp = tmp.sort_values(["Player", "_season_order"]).copy()
            tmp["MarketValue_NextSeason"] = tmp.groupby("Player")["MarketValueEuro"].shift(-1)
            df_with_market_values = tmp.drop(columns=["_season_order"])  # type: ignore[assignment]
        else:
            raise ValueError("Missing 'MarketValue_NextSeason' and cannot derive it.")
    df_clean: pd.DataFrame = df_with_market_values.dropna(subset=["MarketValue_NextSeason"]).copy()
    return df_clean


def select_features(df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.Series, List[str]]:
    """Keep numeric features, drop high-cardinality identifiers."""
    columns_to_drop: List[str] = ["Player", "Pos", "Primary_Pos", "Pos_Category", "Nationality"]
    df_model: pd.DataFrame = df.drop(columns=[c for c in columns_to_drop if c in df.columns]).copy()
    numeric_df: pd.DataFrame = df_model.select_dtypes(include=[np.number]).copy()
    y: pd.Series = numeric_df["MarketValue_NextSeason"].astype(float)
    X: pd.DataFrame = numeric_df.drop(columns=["MarketValue_NextSeason"]).astype(float)
    feature_names: List[str] = list(X.columns)
    return X, y, feature_names

In [7]:
def kfold_indices(n_samples: int, n_splits: int = 5) -> List[Tuple[np.ndarray, np.ndarray]]:
    """Return list of (train_idx, valid_idx) for KFold with shuffling."""
    kf: KFold = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    return list(kf.split(np.arange(n_samples)))


In [8]:
def objective_factory(X: pd.DataFrame, y: pd.Series, folds: List[Tuple[np.ndarray, np.ndarray]]):
    """Create an Optuna objective that performs KFold CV with XGBoost."""
    def objective(trial: optuna.trial.Trial) -> float:
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 200, 2000, step=100),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "min_child_weight": trial.suggest_float("min_child_weight", 1e-2, 10.0, log=True),
            "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 1e-1, log=True),
            "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 100.0, log=True),
            "gamma": trial.suggest_float("gamma", 0.0, 5.0),
            "objective": "reg:squarederror",
            "random_state": RANDOM_SEED,
            "tree_method": "hist",
            "n_jobs": -1,
            "eval_metric": "mae",
        }

        fold_maes: List[float] = []
        for train_idx, valid_idx in folds:
            X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
            X_valid, y_valid = X.iloc[valid_idx], y.iloc[valid_idx]

            model = XGBRegressor(**params)
            model.fit(
                X_train,
                y_train,
                eval_set=[(X_valid, y_valid)],
                verbose=False,
            )
            y_pred = model.predict(X_valid)
            fold_maes.append(mean_absolute_error(y_valid, y_pred))

        return float(np.mean(fold_maes))

    return objective

In [9]:
def train_final_model(X: pd.DataFrame, y: pd.Series, best_params: Dict[str, object]) -> XGBRegressor:
    """Train XGBoost on full data with best params and return fitted model."""
    final_params = {**best_params}
    final_params.update({
        "objective": "reg:squarederror",
        "random_state": RANDOM_SEED,
        "tree_method": "hist",
        "n_jobs": -1,
        "eval_metric": "mae",
    })
    model = XGBRegressor(**final_params)
    model.fit(X, y, verbose=False)
    return model


def save_artifacts(model: XGBRegressor, study: optuna.Study, feature_names: List[str]) -> None:
    """Persist model, study results and metadata to disk."""
    model_path: Path = MODELS_DIR / "xgb_market_value_next_season.pkl"
    with open(model_path, "wb") as f:
        pickle.dump(model, f)

    study_json_path: Path = ARTIFACTS_DIR / "optuna_xgb_study.json"
    with open(study_json_path, "w") as f:
        json.dump({
            "best_value": study.best_value,
            "best_params": study.best_trial.params,
            "n_trials": len(study.trials),
        }, f, indent=2)

    meta_path: Path = ARTIFACTS_DIR / "xgb_feature_names.json"
    with open(meta_path, "w") as f:
        json.dump({"feature_names": feature_names}, f, indent=2)

In [10]:
from typing import TypedDict

class FoldMetrics(TypedDict):
    mae: float
    rmse: float
    r2: float


def cross_validated_metrics(X: pd.DataFrame, y: pd.Series, folds: List[Tuple[np.ndarray, np.ndarray]], best_params: Dict[str, object]) -> Dict[str, float]:
    """Compute MAE, RMSE, R2 across folds using best params."""
    perf: List[FoldMetrics] = []

    eval_params = {**best_params}
    eval_params.update({
        "objective": "reg:squarederror",
        "random_state": RANDOM_SEED,
        "tree_method": "hist",
        "n_jobs": -1,
        "eval_metric": "mae",
        "early_stopping_rounds": 10,
    })

    for train_idx, valid_idx in folds:
        model = XGBRegressor(**eval_params)
        model.fit(X.iloc[train_idx], y.iloc[train_idx], verbose=False)
        y_pred = model.predict(X.iloc[valid_idx])
        perf.append({
            "mae": mean_absolute_error(y.iloc[valid_idx], y_pred),
            "rmse": float(np.sqrt(mean_squared_error(y.iloc[valid_idx], y_pred))),
            "r2": r2_score(y.iloc[valid_idx], y_pred),
        })

    avg_mae: float = float(np.mean([m["mae"] for m in perf]))
    avg_rmse: float = float(np.mean([m["rmse"] for m in perf]))
    avg_r2: float = float(np.mean([m["r2"] for m in perf]))

    return {"mae": avg_mae, "rmse": avg_rmse, "r2": avg_r2}


In [11]:
df_model: pd.DataFrame = build_target(df_raw)
X, y, feature_names = select_features(df_model)
folds = kfold_indices(n_samples=len(X), n_splits=5)

In [12]:
sampler = TPESampler(seed=RANDOM_SEED)
study = optuna.create_study(direction="minimize", sampler=sampler)
objective = objective_factory(X, y, folds)
study.optimize(objective, n_trials=50, show_progress_bar=True)

[I 2025-08-11 12:28:15,489] A new study created in memory with name: no-name-78f9ca7b-93d6-41e7-87b0-9801b5ec4e9b
Best trial: 0. Best value: 8.14408e+06:   2%|▏         | 1/50 [00:18<15:06, 18.50s/it]

[I 2025-08-11 12:28:33,999] Trial 0 finished with value: 8144080.330718597 and parameters: {'n_estimators': 900, 'max_depth': 12, 'learning_rate': 0.06504856968981275, 'subsample': 0.7993292420985183, 'colsample_bytree': 0.5780093202212182, 'min_child_weight': 0.029375384576328302, 'reg_alpha': 2.5502648504032812e-08, 'reg_lambda': 21.42302175774107, 'gamma': 3.005575058716044}. Best is trial 0 with value: 8144080.330718597.


Best trial: 1. Best value: 7.3822e+06:   4%|▍         | 2/50 [00:29<11:03, 13.82s/it] 

[I 2025-08-11 12:28:44,549] Trial 1 finished with value: 7382197.970323068 and parameters: {'n_estimators': 1500, 'max_depth': 3, 'learning_rate': 0.2526878207508456, 'subsample': 0.9162213204002109, 'colsample_bytree': 0.6061695553391381, 'min_child_weight': 0.035113563139704075, 'reg_alpha': 1.9223460470643606e-07, 'reg_lambda': 0.033205591037519584, 'gamma': 2.6237821581611893}. Best is trial 1 with value: 7382197.970323068.


Best trial: 1. Best value: 7.3822e+06:   6%|▌         | 3/50 [00:39<09:27, 12.07s/it]

...

[I 2025-08-11 12:37:43,755] Trial 47 finished with value: 7048618.435567632 and parameters: {'n_estimators': 1000, 'max_depth': 4, 'learning_rate': 0.0034836614678764393, 'subsample': 0.7002520389887711, 'colsample_bytree': 0.9739525364613145, 'min_child_weight': 0.2514647224710321, 'reg_alpha': 0.05013288141101887, 'reg_lambda': 1.0202574371202406, 'gamma': 0.4405466090500113}. Best is trial 42 with value: 6572581.6545440825.


Best trial: 42. Best value: 6.57258e+06:  98%|█████████▊| 49/50 [09:47<00:15, 15.99s/it]

[I 2025-08-11 12:38:02,847] Trial 48 finished with value: 7105931.820821257 and parameters: {'n_estimators': 1900, 'max_depth': 5, 'learning_rate': 0.0077574337695854555, 'subsample': 0.7562090875828404, 'colsample_bytree': 0.64492282707787, 'min_child_weight': 0.38758301734107914, 'reg_alpha': 0.003033118490411531, 'reg_lambda': 0.026999084368983992, 'gamma': 0.8257716920494361}. Best is trial 42 with value: 6572581.6545440825.


Best trial: 42. Best value: 6.57258e+06: 100%|██████████| 50/50 [09:58<00:00, 11.96s/it]

[I 2025-08-11 12:38:13,511] Trial 49 finished with value: 6833079.459625604 and parameters: {'n_estimators': 1500, 'max_depth': 3, 'learning_rate': 0.011415145736001199, 'subsample': 0.8484256851780656, 'colsample_bytree': 0.9251254441667912, 'min_child_weight': 0.08335698975644734, 'reg_alpha': 0.0014009521965033978, 'reg_lambda': 0.16707387465175502, 'gamma': 1.2567776497341505}. Best is trial 42 with value: 6572581.6545440825.


In [13]:
best_params = study.best_trial.params
final_model = train_final_model(X, y, best_params)
save_artifacts(final_model, study, feature_names)
print("Best Optuna MAE:", study.best_value)
print("Best Params:")
print(json.dumps(best_params, indent=2))

Best Optuna MAE: 6572581.6545440825
Best Params:
{
  "n_estimators": 1300,
  "max_depth": 6,
  "learning_rate": 0.007446888956149069,
  "subsample": 0.7330540872139371,
  "colsample_bytree": 0.9671773327433728,
  "min_child_weight": 0.10611568276980807,
  "reg_alpha": 0.004924721165808119,
  "reg_lambda": 0.18291866793554443,
  "gamma": 0.49756448892703137
}
